## .

In [1]:
#!pip install tqdm

In [2]:
# Ignore the warnings
import warnings
# warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# System related and data input controls
import os

# Data manipulation and visualization
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows = 20
pd.options.display.max_columns = 20
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing

# Modeling algorithms
# General
import statsmodels.api as sm
from scipy import stats

# Model selection
from sklearn.model_selection import train_test_split

# Evaluation metrics
from sklearn import metrics
# for classification
from sklearn.metrics import confusion_matrix, classification_report 
from sklearn.metrics import roc_curve, auc, precision_recall_curve

# 한글 패치
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/malgun.ttf"
font = font_manager.FontProperties(fname = font_path).get_name()
rc('font', family = font)

# 결측값 시각화 모듈
import missingno as msno

from tqdm import tqdm
import itertools
from itertools import product, permutations, combinations

In [3]:
df_dict = dict()
df_columns = pd.read_excel(os.path.join('.', 'Data', '평생교육실태조사_코드명통합_수정본.xlsx')).iloc[:,2:]
df_columns

,항목명_2022,항목명_2021,항목명_2020,항목명_2019,항목명_2018
0,ID,ID,ID,ID,ID
1,총 가구원 수,총 가구원 수,NaN,NaN,NaN
2,조사대상 가구원 수,조사대상 가구원 수,NaN,NaN,NaN
3,조사완료 가구원 수,조사완료 가구원 수,NaN,NaN,NaN
4,문A1-1) 작년(21.01.01~21.12.31) (1) 학위(졸업장) 취득을 위...,문A1)작년(20.01.01~20.12.31)에 (1)학위취득을 위한 교육에 참여한...,문A1.귀하께서는 지난 1년간 졸업장이나 학위취득을 위해 교육을 받은 적이 있습니까?,문A1.귀하께서는 지난 1년 간(2018년 7월 - 2019년 6월) 졸업장이나 학...,문A1.귀하께서는 지난 1년 간(2017년 7월 - 2018년 6월) 졸업장이나 학...
...,...,...,...,...,...
432,평생학습 참여자,평생학습 참여자,평생학습 참여자,지난 1년(2018년 7월 ~ 2019년 6월)동안 평생학습(형식 또는 비형식)에 ...,지난 1년(2017년 7월 ~ 2018년 6월)동안 평생학습(형식 또는 비형식)에 ...
433,형식교육 참여자,형식교육 참여자,형식교육 참여자,지난 1년(2018년 7월 ~ 2019년 6월)동안 형식교육에 참여한 사람,지난 1년(2017년 7월 ~ 2018년 6월)동안 형식교육에 참여한 사람
434,비형식교육 참여자,비형식교육 참여자,비형식교육 참여자,지난 1년(2018년 7월 ~ 2019년 6월)동안 비형식교육에 참여한 사람,지난 1년(2017년 7월 ~ 2018년 6월)동안 비형식교육에 참여한 사람
435,직업관련 비형식교육 참여자,NaN,NaN,지난 1년(2018년 7월 ~ 2019년 6월)동안 직업 관련된 목적으로 비형식교육...,지난 1년(2017년 7월 ~ 2018년 6월)동안 직업 관련된 목적으로 비형식교육...


In [4]:
DF_YEAR = [2022, 2021, 2020, 2019, 2018]
for year in DF_YEAR:
    folder_location=os.path.join('.', 'data', '{}_총괄_20250320_36011.csv'.format(year))
    df_dict[year] = pd.read_csv(folder_location, encoding='cp949')
    print(year, df_dict[year].shape)
    ## 변수명 DF_YEAR[0]기준으로 동일하게 변경
    if year != DF_YEAR[0]:
        df_sub = pd.concat([df_columns[[col for col in df_columns.columns if col.split('_')[1] == str(year)]], df_columns.iloc[:,[0]]], axis=1)
        df_rename = dict(df_sub[df_sub.iloc[:,1] != 0].dropna().reset_index().iloc[:,1:].values)
        df_dict[year] = df_dict[year].rename(columns=df_rename)
      

2022 (9968, 437)
2021 (9905, 454)
2020 (9776, 434)
2019 (9973, 511)
2018 (11747, 600)


In [5]:
display(df_dict)

{2022:            ID  총 가구원 수  조사대상 가구원 수  조사완료 가구원 수  \
 0     1000601        3           2           2   
 1     1000602        3           2           2   
 2     1001901        3           2           2   
 3     1001902        3           2           2   
 4     1002701        3           3           3   
 ...       ...      ...         ...         ...   
 9963   996801        3           2           2   
 9964   996802        3           2           2   
 9965   997901        1           1           1   
 9966   998801        2           2           2   
 9967   998802        2           2           2   
 
       문A1-1) 작년(21.01.01~21.12.31) (1) 학위(졸업장) 취득을 위한 교육 참여 경험 여부  \
 0                                                     2             
 1                                                     2             
 2                                                     2             
 3                                                     2             
 4                            

## Data Merge

In [6]:
def dict_to_concatdf(df_dict):
    df_concat = pd.DataFrame()
    for key, df in df_dict.items():
        df['key'] = key  # key 컬럼 추가
        df_concat = pd.concat([df_concat, df], axis=0)
    return df_concat.reset_index(drop=True)

In [7]:
# 1. 기준 연도 및 데이터프레임 딕셔너리 초기화
DF_YEAR = [2022, 2021, 2020, 2019, 2018]
df_dict = dict()

# 2. 변수명 매핑 정보 불러오기 (항목명 통합표)
df_columns = pd.read_excel(os.path.join('.', 'Data', '평생교육실태조사_코드명통합_수정본.xlsx')).iloc[:, 2:]

# 3. 연도별 데이터 불러오기 및 변수명 2022 기준으로 통일
for year in DF_YEAR:
    file_path = os.path.join('.', 'data', f'{year}_총괄_20250320_36011.csv')
    df = pd.read_csv(file_path, encoding='cp949')

    # 변수명 매핑 적용 (2022년은 그대로 유지)
    if year != DF_YEAR[0]:
        df_sub = pd.concat([
            df_columns[[col for col in df_columns.columns if col.split('_')[1] == str(year)]],
            df_columns.iloc[:, [0]]
        ], axis=1)

        df_rename = dict(df_sub[df_sub.iloc[:, 1] != 0].dropna().reset_index().iloc[:, 1:].values)
        df = df.rename(columns=df_rename)

    df_dict[year] = df
    print(year, df.shape)

# 4. 기준 컬럼(2022년) 기준으로 누락된 컬럼 보완
base_cols = df_dict[2022].columns
for year, df in df_dict.items():
    missing_cols = [col for col in base_cols if col not in df.columns]
    for col in missing_cols:
        df[col] = pd.NA
    # 컬럼 순서를 기준에 맞게 정렬
    df_dict[year] = df[base_cols]

# 5. 병합 수행
merged_df = dict_to_concatdf(df_dict)

# 6. 저장
# merged_df.to_csv('df_raw.csv', encoding='utf-8-sig')

print("병합 완료. shape:", merged_df.shape)

2022 (9968, 437)
2021 (9905, 454)
2020 (9776, 434)
2019 (9973, 511)
2018 (11747, 600)
병합 완료. shape: (51369, 438)


In [8]:
merged_df

,ID,총 가구원 수,조사대상 가구원 수,조사완료 가구원 수,문A1-1) 작년(21.01.01~21.12.31) (1) 학위(졸업장) 취득을 위한 교육 참여 경험 여부,문A1-1) 작년(21.01.01~21.12.31) (2) 성인기초 및 문자해득교육(문해교육) 참여 경험 여부,문A1-1) 작년(21.01.01~21.12.31) (3) 직업능력향상교육 참여 경험 여부,문A1-1) 작년(21.01.01~21.12.31) (4) 인문교양교육 참여 경험 여부,문A1-1) 작년(21.01.01~21.12.31) (5) 문화예술스포츠교육 참여 경험 여부,문A1-1) 작년(21.01.01~21.12.31) (6) 시민참여교육 참여 경험 여부,...,직업,직장의 규모,근무기간,원가중치,평생학습 참여자,형식교육 참여자,비형식교육 참여자,직업관련 비형식교육 참여자,동시 참여자,key
0,1000601,3,2,2,2,2,2,2,1,2,...,3.00,4.00,2.00,"3,813.13",1,2,1,2,2,2022
1,1000602,3,2,2,2,2,2,2,2,2,...,3.00,3.00,2.00,"3,701.18",2,2,2,2,2,2022
2,1001901,3,2,2,2,2,2,2,2,2,...,5.00,1.00,2.00,"4,421.94",2,2,2,2,2,2022
3,1001902,3,2,2,2,2,2,2,2,2,...,3.00,4.00,3.00,"4,555.69",2,2,2,2,2,2022
4,1002701,3,3,3,1,2,2,2,2,2,...,5.00,1.00,3.00,"3,932.43",1,1,2,2,2,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51364,99941,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,...,6.00,1.00,6.00,"1,956.89",2,2,2,2,2,2018
51365,99951,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,...,6.00,1.00,6.00,"1,956.89",2,2,2,2,2,2018
51366,99961,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,...,6.00,1.00,6.00,"2,378.59",2,2,2,2,2,2018
51367,99971,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,...,2.00,5.00,1.00,"3,769.61",2,2,2,2,2,2018
